In [1]:
import gurobipy   as     gp
import pandas as pd
from   gurobipy   import GRB
import subprocess as sp
from globalFunctions import getNumberOfBlocksInADimension
from openPitFunctions import finalBlock

In [3]:
#openPitDataframe = pd.read_excel("C:/Users/willi/OneDrive/Escritorio/Magister/Tesis-Magister/Database/integratedModel/Modelo_F_OG_4_4_4.xlsx", engine="openpyxl")#Notebook
#openPitDataframe = pd.read_excel("C:/Users/Williams Medina/Desktop/Tesis Magister/Tesis-Magister/ThesisCode/MainCode/Databases/integratedModel/Modelo_F_OG_4_4_4.xlsx",engine="openpyxl")#Desktop
path = "/home/williams/Tesis-Magister/Database/integratedModel/"
#openPitDatabaseName = 'Modelo_F_OG.xlsx'
openPitDatabaseName = "Modelo_F_OG_4_4_4.xlsx"
openPitDataframe = pd.read_excel(path + openPitDatabaseName, engine="openpyxl") #Notebook

In [4]:
#Falta agregar la restricción GQC_Up_OP y GQC_low_OP
#Dejar como indice, valor (formulario de función objetivo), duracion,tonelaje bloque, tonelaje * % Cu
#./omp.sh /home/williams/Tesis-Magister/ThesisCode/BendersCode/openPit/files/openPit.* /home/williams/Tesis-Magister/ThesisCode/BendersCode/openPit/params/openPit.params
#./omp.sh ./openPit/files/openPit.* ./openPit/params/openPit.params

#1 periodo como defecto

In [5]:
class OpenPitModel:
   def __init__(self, database, numberOfPeriods):
      self.database = database
      self.numberOfPeriods = numberOfPeriods
      self.numberOfDestinations = 1
      self.basePrice = 3791.912
      self.desc = 0.1

   def execute(self):
      self.setOpenPitVariables()
      self.getOpenPitInfo()
      self.setOpenPitParameters()
      self.setOpenPitMineLimits()
      self.createOmpInput()
      self.executeOmp()
      #self.setFinalParameters()

   def setOpenPitVariables(self):
      self.openPitBlocksLength = self.database['X'].to_dict() 
      self.openPitBlocksWidth = self.database['Y'].to_dict() 
      self.openPitBlocksHeight = self.database['Z'].to_dict() 
      self.L_b = self.database['Ton'].to_dict() #openPitBlockTonnage
      self.o_b = self.database['Mineral'].to_dict() #openPitBlockMineral
      self.openPitBlockRecovery = self.database['Recuperación'].to_dict() #openPitBlockRecovery
      self.openPitCopperLaw = self.database['%Cu'].to_dict() #openPitCopperLaw
      self.c_pbt = self.database['CPlanta CA'].to_dict() #openPitPlantCapacity
      self.c_mbt = self.database['CMina CA'].to_dict() #openPitMineCapacity

   def getOpenPitInfo(self):
      self.openPitBlocks = [i for i in range(len(self.openPitBlocksLength.values()))]

   def setOpenPitParameters(self):
      #OpenPit Parameters
      self.t_C   = {period : period + 1 for period in range(self.numberOfPeriods)}
      self.RMu_t = {period : 13219200.0 for period in range(self.numberOfPeriods)}#Superior infinita, 0 por abajo Originales: 13219200
      self.RMl_t = {period : 8812800.0 for period in range(self.numberOfPeriods)}#Valor original 8812800.0
      self.RPu_t = {period : 10933380.0 for period in range(self.numberOfPeriods)}#Valor original 10933380.0
      self.RPl_t = {period : 0 for period in range(self.numberOfPeriods)}#Valor original 7288920.0 
      self.qu_t  = {period : 1 for period in range(self.numberOfPeriods)}#Leyes promedio maxima y minima.
      self.ql_t  = {period : 0 for period in range(self.numberOfPeriods)}
      self.maxTimeOpenPit = self.t_C[max(self.t_C)]

   def setOpenPitMineLimits(self):
      self.openPitBlocksLengthLimits = getNumberOfBlocksInADimension(self.openPitBlocksLength)
      self.openPitBlocksWidthLimits = getNumberOfBlocksInADimension(self.openPitBlocksWidth)
      self.openPitBlocksHeightLimits = getNumberOfBlocksInADimension(self.openPitBlocksHeight)
      self.predecessorBlock = self.setPredecessorBlocks()

   def setModelAndGetResults(self):
      self.objValue, self.variableValues, self.runtime, self.gap = self.openPitModel()

   def setPredecessorBlocks(self):
      self.predecessorBlocks = finalBlock(self.openPitBlocks, self.openPitBlocksLengthLimits,self.openPitBlocksWidthLimits, self.openPitBlocksHeightLimits)

   def createOmpInput(self):
      self.writeProblemFile()
      self.writeBlocksFile()
      self.writePrecFile()
      self.writeParamsFile()
   
   def writeProblemFile(self):
      with open('openPit/files/openPit.prob', 'w') as f:
         numberOfDestinations = 'NDESTINATIONS: ' + str(self.numberOfDestinations)
         numberOfPeriods = 'NPERIODS: ' + str(self.numberOfPeriods)
         objective = 'OBJECTIVE: 0 1'
         duration = 'DURATION: 2'
         discountRate = 'DISCOUNT_RATE: '+ str(self.desc)#/self.numberOfPeriods)
         numberOfConstraints = 'NCONSTRAINTS: 6'
         f.write('{}\n{}\n{}\n{}\n{}\n{}\n'.format(numberOfDestinations,numberOfPeriods, objective,duration,discountRate,numberOfConstraints))
         tonUpConstraint = 'CONSTRAINT: 0 3 P * L '
         for rmu in self.RMu_t.values():
            tonUpConstraint +=str(rmu) + " "

         tonLowContraint = 'CONSTRAINT: 1 3 P * G '
         for rml in self.RMl_t.values():
            tonLowContraint +=str(rml) + " "
         
         matUpConstraint = 'CONSTRAINT: 2 4 P * L '
         for rpu in self.RPu_t.values():
            matUpConstraint +=str(rpu) + " "

         matLowConstraint = 'CONSTRAINT: 3 4 P * G '
         for rpl in self.RPl_t.values():
            matLowConstraint +=str(rpl) + " "
         
         copperLawUpConstraint = 'CONSTRAINT: 4 5 P * L '
         for qut in self.qu_t.values():
            copperLawUpConstraint +=str(qut*sum(self.L_b.values())) + " "

         copperLawLowConstraint = 'CONSTRAINT: 5 5 P * G '
         for qlt in self.ql_t.values():
            copperLawLowConstraint +=str(qlt*sum(self.L_b.values())) + " "
         f.write('{}\n{}\n{}\n{}\n{}\n{}\n'.format(tonUpConstraint, tonLowContraint, matUpConstraint, matLowConstraint, copperLawUpConstraint,copperLawLowConstraint))

   def writeBlocksFile(self):
      with open('openPit/files/openPit.blocks', 'w') as f:
         for block in self.openPitBlocks:
            index = block
            value = ((self.basePrice*self.openPitCopperLaw[block]-self.c_pbt[block])*self.o_b[block])-(self.c_mbt[block]*self.L_b[block])
            duration = 1
            ton = self.L_b[block]
            mineral = self.o_b[block]
            copperLaw = self.openPitCopperLaw[block] * self.L_b[block]
            f.write(('{} {} {} {} {} {}\n').format(index, value, duration,ton, mineral,copperLaw))
                 
   def writePrecFile(self):
      with open('openPit/files/openPit.prec', 'w') as f:
         for index,blockList in enumerate(self.predecessorBlocks):
            predecessorLine = str(len(blockList))
            for block in blockList:
               if block == index:
                  predecessorLine = " 0"
                  break
               else:
                  predecessorLine +=" " + str(block)
            f.write("{} {}\n".format(index, predecessorLine))
            
   def writeParamsFile(self):
      with open('openPit/params/openPit.params', 'w') as f:
         f.write("""USE_DISPLAY: 1\n 
                  WRITE.LP.SOLUTION: 0\n
                  WRITE.IP.SOLUTION: 1\n
                  CPIT: 1\n
                  PP.ULTIMATE_PIT: 1\n
                  PP.FORCE_UPIT: 1\n
                  PP.EARLY_START: 1\n
                  PP.WASTE_OPTION: 1\n
                  PP.ELIM_NULL: 0\n
                  PP.TRANSITIVE_REDUCTION: 0\n
                  AG.USE_BLOCK_AGGREGATION: 0\n
                  AG.BLOCK_AGGREGATION: 1.0\n
                  OPTMETHOD: 0\n
                  CG.ONE_DESTINATION: 1\n
                  CG.IMPLICIT: 0\n
                  CG.USE_DISPLAY: 1\n
                  CG.MAX_ITER: -1\n
                  CG.TARGET_GAP: 0.0001\n
                  CG.MAX_TIME: -1\n
                  CG.USE_KSTEP: 0\n
                  CG.KSTEP_K: 10\n
                  CG.MASTER_NTHREADS: 4\n
                  HE.TOPOSORT: 1\n
                  HE.FTOPOSORT: 0\n
                  HE.NALPHA_POINTS: 50\n
                  HE.OPT_DESTINATIONS: 1\n
                  HE.NAIVE: 0\n
                  HE.NAIVE_INTSOLLIM: 1000\n
                  HE.NAIVE_EPGAP: 0.01\n
                  HE.NAIVE_TILIM: 14400\n
                  CP.DYNAMIC_CUTS: 0\n
                  CP.CLIQUES: 0\n
                  CP.MINW: 0\n
                  CP.DELAYED_PRECEDENCES: 0\n
                  CONSTRAINT_PROGRAMMING: 0\n
                  CPROG.GAP_LIMIT: 0.01\n
                  CPROG.CP_TIME_LIMIT: 28800\n
                  CPROG.EX_TIME_LIMIT: 28800\n
                  CPROG.HOT_START: 1\n
                  CPROG.NTHREADS: 8""")

   def executeOmp(self):
      output = sp.getoutput("./omp.sh /home/williams/Tesis-Magister/ThesisCode/BendersCode/openPit/files/openPit.* /home/williams/Tesis-Magister/ThesisCode/BendersCode/openPit/params/openPit.params")
      print (output)
      print()
      print()
      print("ya termine")

In [6]:
numberOfPeriods = 5
openPitModel = OpenPitModel(openPitDataframe,numberOfPeriods)

In [7]:
openPitModel.execute()

Running OMP version: master 1.8.4 bf856556 2022-08-12 13:50:47 -0400
Invoked using: bin/OMP_SOLVER_C1290 /home/williams/Tesis-Magister/ThesisCode/BendersCode/openPit/files/openPit.blocks /home/williams/Tesis-Magister/ThesisCode/BendersCode/openPit/files/openPit.prec /home/williams/Tesis-Magister/ThesisCode/BendersCode/openPit/files/openPit.prob /home/williams/Tesis-Magister/ThesisCode/BendersCode/openPit/params/openPit.params
license to sign [DEFAULTclient-signatureAFhG-4bsA-AAA=lic_ver200valid-to2023-03-07]
LICENSE OK!
BLOCKS FILE: /home/williams/Tesis-Magister/ThesisCode/BendersCode/openPit/files/openPit.blocks
PREC FILE: /home/williams/Tesis-Magister/ThesisCode/BendersCode/openPit/files/openPit.prec
PROB FILE: /home/williams/Tesis-Magister/ThesisCode/BendersCode/openPit/files/openPit.prob
PARAMS FILE: /home/williams/Tesis-Magister/ThesisCode/BendersCode/openPit/params/openPit.params
OR Solver: key created. The key is: openPit_openPit_openPit
STATS_INPUT_NAME openPit_openPit_openPit


In [8]:
openPitModel.t_C

{0: 1, 1: 2, 2: 3, 3: 4, 4: 5}

In [ ]:
a = ""
for value in openPitModel.RMu_t.values():
    a +=str(value) + " "
a

'13219200.0 13219200.0 13219200.0 13219200.0 13219200.0 '

In [ ]:
a = [str(value) + " " for value in openPitModel.RMu_t.values()]

In [ ]:
a

['13219200.0 ', '13219200.0 ', '13219200.0 ', '13219200.0 ', '13219200.0 ']